# Gather Historical Twitter Data about COVID-19 vaccine

In [2]:
import tweepy as tw
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime
import seaborn as sns
from matplotlib import pyplot as plt
import calendar

In [3]:
# Get this from the app that you create on the developer portal
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAEJSWwEAAAAAW0pE8iFqsh6Z6HMK5SEkqtRbt4E%3D9YcPS68vNQybgeQS2Mv64SKSe8fWbUD5w4iUGUzF1ApIV6LUy0"

In [4]:
# Initialize the API client and tell tweepy to wait if rate limit is met
client = tw.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

In [6]:
# Example start and end date
start_date = datetime.strptime("2021-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2021-12-15", "%Y-%m-%d") 
query_results = []
places = {}
max_results_per_request = 500 # must be between 10 and 500

# Use the paginator to loop over all available tweets in batches of 500
# This pulls 500 tweets at a time until all tweets from the time period are pulled
for i, tweets in enumerate(
    tw.Paginator(
        client.search_all_tweets,
        query="#Covid19vaccine -is:retweet place:USA", #repace with other hastags related to covid vaccines like #COVIDvaccine #moderna #Pfizer etc.
        tweet_fields=["created_at", "geo"],
        place_fields=["place_type", "geo"],
        expansions="geo.place_id",
        max_results=max_results_per_request,
        start_time=start_date,
        end_time=end_date,
        limit=1000 #use with 1000
    )
):
    print(f"Response #{i+1}")
    
    # Store the tweet data
    # Note: 'tweets.data' is a list of the 500 tweets
    query_results += tweets.data
    
        # Keep track of all of the places
    for p in tweets.includes['places']:
        places[p["id"]] = p
    


Response #1


In [7]:
# Convert to a dataframe and make a datetime column
query_results = pd.DataFrame(query_results)
query_results['created_at'] = pd.to_datetime(query_results['created_at'])

# Extract out the place id as its own column
query_results['place_id'] = query_results['geo'].apply(lambda geo: geo['place_id'])

In [8]:
# Convert places dict to a dataframe
places_df = []

# Loop over each place_id
for place_id in places:
    
    # Geo info for each place
    values = places[place_id]
    
    # Convert to a dict
    row = dict(values)
    
    # Save it
    places_df.append(row)
    
# Initialize the dataframe
places_df = pd.DataFrame(places_df).rename(columns={"id": "place_id"})

In [10]:
query_results.head()

,created_at,geo,id,text,place_id
0,2021-12-09 17:00:21+00:00,{'place_id': 'dd9c503d6c35364b'},1468988903516520457,Everyone even if vaccinated should watch these...,dd9c503d6c35364b
1,2021-11-16 22:27:35+00:00,{'place_id': '226b21641df42460'},1460736332553203713,@MemHospitalNH Kudos to this dedicated team! 🏆...,226b21641df42460
2,2021-11-14 18:44:47+00:00,{'place_id': 'dd9c503d6c35364b'},1459955488158892043,"Mandate or no mandate, getting a #COVID19Vacci...",dd9c503d6c35364b
3,2021-11-12 23:59:01+00:00,{'place_id': '43d2418301bf1a49'},1459309789519962116,https://t.co/ysXpCGuMra @BillGates @CDCgov #CO...,43d2418301bf1a49
4,2021-11-10 05:41:29+00:00,{'place_id': 'fbd6d2f5a4e4a15e'},1458308811970600961,FREE #COVID19Vaccine clinic @DeckerPUSD on Su...,fbd6d2f5a4e4a15e


In [11]:
places_df.head()

,full_name,geo,place_id,place_type
0,"Pennsylvania, USA","{'type': 'Feature', 'bbox': [-80.519851, 39.71...",dd9c503d6c35364b,admin
1,"New Hampshire, USA","{'type': 'Feature', 'bbox': [-72.557247, 42.69...",226b21641df42460,admin
2,"Mississippi, USA","{'type': 'Feature', 'bbox': [-91.655009, 30.14...",43d2418301bf1a49,admin
3,"California, USA","{'type': 'Feature', 'bbox': [-124.482003, 32.5...",fbd6d2f5a4e4a15e,admin
4,"Texas, USA","{'type': 'Feature', 'bbox': [-106.645646, 25.8...",e0060cda70f5f341,admin


In [12]:
# Merge in the places geo data and the query results
query_results_final = query_results.merge(places_df, how='left', on='place_id')
query_results_final=query_results_final.drop(['geo_x', 'id','place_id','geo_y','place_type'], axis=1)
query_results_final['created_at']=query_results_final['created_at'].dt.strftime('%d-%m-%Y')

In [22]:
query_results_final.head()

,created_at,text,full_name
0,30-03-2021,I have been vaxxed! Start the clock for the se...,"Florida, USA"
1,30-03-2021,1st 💉 down. 1 to go. Big shout out to @ACHealt...,"North Carolina, USA"
2,30-03-2021,"My son-in-law was among the 13,000 today. So g...","Michigan, USA"
3,29-03-2021,"Got dose #1 today, looking forward to dose #2 ...","Ohio, USA"
4,29-03-2021,@POTUS @CityFoodLover such a great news! 90% o...,"Florida, USA"


In [33]:
#rename the file starting with 'his' and store it as csv
query_results_final.to_csv('his.csv',index=False,encoding='utf-8')